In [ ]:
import os
import numpy as np
import soundfile as sf
import scipy.signal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import Sequence
import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
from scipy.ndimage import zoom
import ctypes
from kerastuner.tuners import RandomSearch
import scipy.fftpack

# Set GPU configuration (if applicable)
gpus = tf.config.experimental.list_physical_devices('GPU') 
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


2024-11-22 09:18:57.439994: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-22 09:18:57.746921: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-22 09:18:57.746950: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-22 09:18:57.748804: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 09:18:57.908329: I tensorflow/core/platform/cpu_feature_g

Num GPUs Available:  1


2024-11-22 09:19:00.704922: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-22 09:19:00.704964: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
AUDIO_PATH = '/home/garfield/CryCorpusFinal'
CRY_FOLDER = os.path.join(AUDIO_PATH, 'cry_augmented')
NOTCRY_FOLDER = os.path.join(AUDIO_PATH, 'notcry_augmented')
NUM_MFCC = 20  #  Number of MFCC coefficients to extract
BATCH_SIZE = 32
EPOCHS = 50
MODEL = 'cnn'  # Choice: 'cnn' or 'lstm'


In [ ]:


def load_audio_files(folder):
    files = []
    for filename in os.listdir(folder):
        if filename.endswith('.wav'):
            files.append(os.path.join(folder, filename))
    return files

def compute_mfcc(y, sr, n_mfcc=NUM_MFCC):
    # Pre-emphasis filter
    pre_emphasis = 0.97
    emphasized_signal = np.append(y[0], y[1:] - pre_emphasis * y[:-1])
    
    # Frame parameters
    frame_size = 0.025  # 25 ms
    frame_stride = 0.010  # 10 ms
    frame_length, frame_step = int(round(frame_size * sr)), int(round(frame_stride * sr))
    signal_length = len(emphasized_signal)
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))
    
    # Padding
    pad_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(emphasized_signal, z)
    
    # Framing
    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + \
              np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]
    
    # Windowing with Hamming window
    frames *= np.hamming(frame_length)
    
    # Fourier Transform and Power Spectrum
    NFFT = 512
    mag_frames = np.absolute(np.fft.rfft(frames, NFFT))  # Magnitude spectrum
    pow_frames = ((1.0 / NFFT) * (mag_frames ** 2))  # Power spectrum
    
    # Filter Banks
    nfilt = 40
    low_freq_mel = 0
    high_freq_mel = (2595 * np.log10(1 + (sr / 2) / 700))  # Convert Hz to Mel
    mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Mel scale
    hz_points = (700 * (10 ** (mel_points / 2595) - 1))  # Mel to Hz
    bin = np.floor((NFFT + 1) * hz_points / sr).astype(int)
    
    fbank = np.zeros((nfilt, NFFT // 2 + 1))
    for m in range(1, nfilt + 1):
        f_m_minus = bin[m - 1]   # Left
        f_m = bin[m]             # Center
        f_m_plus = bin[m + 1]    # Right
        
        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
    
    filter_banks = np.dot(pow_frames, fbank.T)
    # Numerical stability
    filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)
    filter_banks = 20 * np.log10(filter_banks)  # Convert to dB
    
    # MFCCs
    mfcc = scipy.fftpack.dct(filter_banks, type=2, axis=1, norm='ortho')[:, :n_mfcc]
    # Mean normalization
    mfcc -= (np.mean(mfcc, axis=0) + 1e-8)
    
    return mfcc

def save_mfcc_to_disk(mfcc, save_path):
    if not os.path.exists(os.path.dirname(save_path)):
        os.makedirs(os.path.dirname(save_path))
    np.save(save_path, mfcc)


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# # Collect sequence lengths
# sequence_lengths = []
# for file_path in data:
#     mfcc = np.load(file_path)
#     sequence_lengths.append(mfcc.shape[0])

# sequence_lengths = np.array(sequence_lengths)

# # Plot histogram
# plt.hist(sequence_lengths, bins=50)
# plt.xlabel('Sequence Length (Number of Frames)')
# plt.ylabel('Number of Sequences')
# plt.title('Distribution of MFCC Sequence Lengths')
# plt.show()

# # Calculate percentiles
# print("Maximum sequence length:", np.max(sequence_lengths))
# print("95th percentile length:", np.percentile(sequence_lengths, 95))
# print("90th percentile length:", np.percentile(sequence_lengths, 90))
# print("Median sequence length:", np.median(sequence_lengths))




In [ ]:
cry_files = load_audio_files(CRY_FOLDER)
notcry_files = load_audio_files(NOTCRY_FOLDER)

data = []
labels = []

for idx, file in enumerate(cry_files):
    y, sr = sf.read(file)
    y = y.astype(np.float32)
    # Normalize the audio signal
    y = y / np.max(np.abs(y))
    mfcc = compute_mfcc(y, sr, n_mfcc=NUM_MFCC)
    save_path = os.path.join(f'{AUDIO_PATH}/mfccs', f'cry_{idx}.npy')
    save_mfcc_to_disk(mfcc, save_path)
    data.append(save_path)
    labels.append(1)
    del y, mfcc  # Free up memory

for idx, file in enumerate(notcry_files):
    y, sr = sf.read(file)
    y = y.astype(np.float32)
    # Normalize the audio signal
    y = y / np.max(np.abs(y))
    mfcc = compute_mfcc(y, sr, n_mfcc=NUM_MFCC)
    save_path = os.path.join(f'{AUDIO_PATH}/mfccs', f'notcry_{idx}.npy')
    save_mfcc_to_disk(mfcc, save_path)
    data.append(save_path)
    labels.append(0)
    del y, mfcc  # Free up memory

data = np.array(data)
labels = np.array(labels)


In [ ]:
# Split data
X_train, X_val, y_train, y_val = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

# Set the fixed maximum sequence length
MAX_LENGTH = 499

class OnTheFlyDataGenerator(tf.keras.utils.Sequence):
    def __init__(
        self, file_paths, labels, batch_size, num_mfcc, max_length, shuffle=True, augment=False, is_lstm=False
    ):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.num_mfcc = num_mfcc
        self.max_length = max_length
        self.shuffle = shuffle
        self.augment = augment
        self.is_lstm = is_lstm
        self.indices = np.arange(len(self.file_paths))
        self.on_epoch_end()
        self.cache = {}

    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[
            index * self.batch_size : min((index + 1) * self.batch_size, len(self.file_paths))
        ]
        batch_file_paths = [self.file_paths[i] for i in batch_indices]
        batch_labels = [self.labels[i] for i in batch_indices]

        X, y = self.__data_generation(batch_file_paths, batch_labels)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, batch_file_paths, batch_labels):
        X = []
        y = []

        for i, file_path in enumerate(batch_file_paths):
            mfcc = np.load(file_path)

            if self.augment:
                mfcc = self._augment_mfcc(mfcc)

            if mfcc.shape[0] < self.max_length:
                pad_width = ((0, self.max_length - mfcc.shape[0]), (0, 0))
                mfcc = np.pad(mfcc, pad_width, mode='constant')
            else:
                mfcc = mfcc[: self.max_length, :]

            X.append(mfcc)
            y.append(batch_labels[i])
            del mfcc

        X = np.array(X)
        y = np.array(y)

        if self.is_lstm:
            return X, y
        else:
            X = X[..., np.newaxis]
            return X, y

    def _augment_mfcc(self, mfcc):
        noise_factor = 0.005
        noise = np.random.randn(*mfcc.shape)
        mfcc += noise_factor * noise

        time_shift = np.roll(mfcc, shift=np.random.randint(-10, 10), axis=0)

        mfcc = np.clip(mfcc, -1, 1)
        return time_shift


# CNN Model with Improved Regularization and Gradient Clipping
def build_cnn_model(hp):
    l2_regularizer = tf.keras.regularizers.l2(
        hp.Float('l2', min_value=1e-4, max_value=1e-2, sampling='LOG')
    )
    model = Sequential()

    # Reduced model complexity
    model.add(
        Conv2D(
            filters=hp.Int('filters_1', min_value=32, max_value=96, step=32),
            kernel_size=(3, 3),
            activation='relu',
            padding='same',
            input_shape=(MAX_LENGTH, NUM_MFCC, 1),
            kernel_regularizer=l2_regularizer,
        )
    )
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.3, max_value=0.6, step=0.1)))

    model.add(
        Conv2D(
            filters=hp.Int('filters_2', min_value=64, max_value=128, step=64),
            kernel_size=(3, 3),
            activation='relu',
            padding='same',
            kernel_regularizer=l2_regularizer,
        )
    )
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.3, max_value=0.6, step=0.1)))

    model.add(Flatten())
    model.add(
        Dense(
            units=hp.Int('dense_units', min_value=64, max_value=128, step=64),
            activation='relu',
            kernel_regularizer=l2_regularizer,
        )
    )
    model.add(Dropout(rate=hp.Float('dropout_fc', min_value=0.3, max_value=0.6, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(
        learning_rate=hp.Float(
            'learning_rate', min_value=1e-5, max_value=1e-3, sampling='LOG'
        )
    )
    optimizer = tf.keras.optimizers.Adam(clipnorm=1.0)  # Gradient clipping
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy'],
    )
    model.summary()
    return model


# LSTM Model with Improvements
def build_lstm_model(hp):
    model = Sequential()

    # LSTM layers with regularization
    model.add(
        LSTM(
            units=hp.Int('lstm_units_1', min_value=64, max_value=256, step=64),
            input_shape=(MAX_LENGTH, NUM_MFCC),
            return_sequences=True,
            kernel_regularizer=tf.keras.regularizers.l2(
                hp.Float('l2', min_value=1e-4, max_value=1e-2, sampling='LOG')
            ),
        )
    )
    model.add(
        Dropout(
            rate=hp.Float('dropout_lstm_1', min_value=0.3, max_value=0.6, step=0.1)
        )
    )

    model.add(
        LSTM(
            units=hp.Int('lstm_units_2', min_value=64, max_value=256, step=64),
            return_sequences=True,
        )
    )
    model.add(
        Dropout(
            rate=hp.Float('dropout_lstm_2', min_value=0.3, max_value=0.6, step=0.1)
        )
    )

    model.add(
        LSTM(
            units=hp.Int('lstm_units_3', min_value=64, max_value=256, step=64),
            return_sequences=False,
        )
    )
    model.add(
        Dropout(
            rate=hp.Float('dropout_lstm_3', min_value=0.3, max_value=0.6, step=0.1)
        )
    )

    # Fully connected layers
    model.add(
        Dense(
            units=hp.Int('dense_units', min_value=64, max_value=128, step=64),
            activation='relu',
            kernel_regularizer=tf.keras.regularizers.l2(
                hp.Float('l2', min_value=1e-4, max_value=1e-2, sampling='LOG')
            ),
        )
    )
    model.add(
        Dropout(rate=hp.Float('dropout_fc', min_value=0.3, max_value=0.6, step=0.1))
    )
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(
        learning_rate=hp.Float(
            'learning_rate', min_value=1e-5, max_value=1e-3, sampling='LOG'
        )
    )
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy'],
    )
    model.summary()
    return model

train_generator = OnTheFlyDataGenerator(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    num_mfcc=NUM_MFCC,
    max_length=MAX_LENGTH,
    shuffle=True,
    augment=True,
    is_lstm=(MODEL == 'lstm'),
)

val_generator = OnTheFlyDataGenerator(
    X_val,
    y_val,
    batch_size=BATCH_SIZE,
    num_mfcc=NUM_MFCC,
    max_length=MAX_LENGTH,
    shuffle=False,
    augment=False,
    is_lstm=(MODEL == 'lstm'),
)

# Training Function
def train_model_with_scheduler(model, model_name):
    log_dir = (
        "logs/fit/"
        + model_name
        + "_"
        + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    )
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    checkpoint_callback = ModelCheckpoint(
        filepath=f'{model_name}_model',
        monitor='val_loss',
        save_best_only=True,
        mode='min',
    )

    lr_callback = ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=25, min_lr=1e-5
    )

    early_stopping_callback = EarlyStopping(
        monitor='val_loss', patience=45, restore_best_weights=True
    )

    history = model.fit(train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        callbacks=[
            tensorboard_callback,
            checkpoint_callback,
            lr_callback,
            early_stopping_callback,
        ],
    )
    return history


# Hyperparameter Tuning and Training
if MODEL == 'cnn':
    tuner = RandomSearch(
        build_cnn_model,
        objective='val_accuracy',
        max_trials=30,
        executions_per_trial=1,
        directory='hyperparam_tuning',
        project_name='cnn_tuning',
    )

    tuner.search(
        train_generator,
        epochs=25,
        validation_data=val_generator,
        callbacks=[EarlyStopping(monitor='val_loss', patience=15)],
    )
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    model = tuner.hypermodel.build(best_hps)
    history = train_model_with_scheduler(model, 'cnn_best')

elif MODEL == 'lstm':
    tuner = RandomSearch(
        build_lstm_model,
        objective='val_accuracy',
        max_trials=30,
        executions_per_trial=1,
        directory='hyperparam_tuning',
        project_name='lstm_tuning',
    )

    tuner.search(
        train_generator,
        epochs=25,
        validation_data=val_generator,
        callbacks=[EarlyStopping(monitor='val_loss', patience=15)],
    )
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    model = tuner.hypermodel.build(best_hps)
    history = train_model_with_scheduler(model, 'lstm_best')

In [ ]:
def load_validation_data(file_paths, labels, num_mfcc, max_length, is_lstm):
    X = []
    y = []
    for i, file_path in enumerate(file_paths):
        mfcc = np.load(file_path)
        # Pad or truncate the MFCC sequence to max_length
        if mfcc.shape[0] < max_length:
            pad_width = ((0, max_length - mfcc.shape[0]), (0, 0))
            mfcc = np.pad(mfcc, pad_width, mode='constant')
        else:
            mfcc = mfcc[:max_length, :]

        if not is_lstm:
            mfcc = mfcc[..., np.newaxis]  # Add channel dimension

        X.append(mfcc)
        y.append(labels[i])

    X = np.array(X)
    y = np.array(y)
    return X, y

def evaluate_and_save_model(model, model_name, X_val_data, y_true, is_lstm=False):
    # Make predictions
    y_pred = model.predict(X_val_data)
    y_pred = (y_pred > 0.5).astype(int)

    # Calculate accuracy and F1 score
    accuracy = accuracy_score(y_true, y_pred)  # Use a distinct name for the variable
    f1 = f1_score(y_true, y_pred)  # Ensure no shadowing of the function name

    # Print the metrics
    print(f'Accuracy for {model_name}: {accuracy:.4f}')
    print(f'F1 Score for {model_name}: {f1:.4f}')

    # Save the model
    model.save(f'{model_name}_cry_detection_model.keras')
    print(f'{model_name} model saved as {model_name}_cry_detection_model.keras')


# Model selection, training, and evaluation
if MODEL == 'cnn':
    # Load validation data
    X_val_data, y_true = load_validation_data(X_val, y_val, NUM_MFCC, MAX_LENGTH, is_lstm=False)
    # Evaluate and save the CNN model
    evaluate_and_save_model(model, 'cnn', X_val_data, y_true, is_lstm=False)

elif MODEL == 'lstm':
    # Load validation data
    X_val_data, y_true = load_validation_data(X_val, y_val, NUM_MFCC, MAX_LENGTH, is_lstm=True)
    # Evaluate and save the LSTM model
    evaluate_and_save_model(model, 'lstm', X_val_data, y_true, is_lstm=True)



In [ ]:
import pathlib

# Create directory for TFLite models
tflite_models_dir = pathlib.Path("tflite_models")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Allow Select TF Ops for both CNN and LSTM models if necessary
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]

# For LSTM models, disable experimental lowering of tensor list ops
if MODEL == 'lstm':
    converter._experimental_lower_tensor_list_ops = False

# Convert the model
tflite_model = converter.convert()

# Save the model
tflite_model_file = tflite_models_dir / "cry_detection_model.tflite"
tflite_model_file.write_bytes(tflite_model)

# Apply optimizations and convert again
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_quant_model = converter.convert()
tflite_model_quant_file = tflite_models_dir / "cry_detection_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

print("TFLite conversion successful!")


In [ ]:
import numpy as np
import soundfile as sf

# Initialize the TFLite interpreter
interpreter = tf.lite.Interpreter(model_path="tflite_models/cry_detection_model_quant.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def preprocess_audio(file_path, num_mfcc=NUM_MFCC, is_lstm=False):
    y, sr = sf.read(file_path)
    y = y.astype(np.float32)
    # Normalize the audio signal
    y = y / np.max(np.abs(y))
    # Compute MFCCs
    mfcc = compute_mfcc(y, sr, n_mfcc=num_mfcc)
    # Add channel dimension for CNN models
    if not is_lstm:
        mfcc = mfcc[..., np.newaxis]
    return mfcc

def predict(file_path, num_mfcc=NUM_MFCC, is_lstm=False):
    input_data = preprocess_audio(file_path, num_mfcc, is_lstm)
    
    # Adjust input shape for LSTM and CNN
    if is_lstm:
        input_data = np.expand_dims(input_data, axis=0).astype(np.float32)  # Shape: (1, time_steps, num_mfcc)
    else:
        input_data = np.expand_dims(input_data, axis=0).astype(np.float32)  # Shape: (1, time_steps, num_mfcc, 1)
    
    # Set the tensor to point to the input data for inference
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # Run inference
    interpreter.invoke()
    
    # Get the output prediction
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    return output_data

def process_folder(folder_path, num_mfcc=NUM_MFCC, is_lstm=False):
    correct_predictions = 0
    total_files = 0
    results = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.wav'):
            file_path = os.path.join(folder_path, file_name)
            prediction = predict(file_path, num_mfcc, is_lstm)
            prediction_label = 'Cry' if prediction > 0.5 else 'Not Cry'
            results.append((file_name, prediction_label))
            ground_truth = 'Cry' if '_cry.wav' in file_name else 'Not Cry'

            if prediction_label == ground_truth:
                correct_predictions += 1

            total_files += 1

    accuracy = (correct_predictions / total_files) * 100 if total_files > 0 else 0

    return results, accuracy


folder_path = '{0}/Test_augmented'.format(AUDIO_PATH)
if MODEL == 'lstm': 
    is_lstm_model = True  
else: 
    is_lstm_model = False
predictions, accuracy = process_folder(folder_path, num_mfcc=NUM_MFCC, is_lstm=is_lstm_model)

for file_name, prediction_label in predictions:
    print(f"File: {file_name}, Prediction: {prediction_label}")

print(f"Prediction Accuracy: {accuracy:.2f}%")


In [5]:
import numpy as np
import soundfile as sf
import scipy.fftpack
import ctypes

# Load the TFLite C library
lib = ctypes.cdll.LoadLibrary('/home/garfield/cry-detection/libtensorflowlite_c_2_14_1_amd64.so')

# Define types for the C API functions
lib.TfLiteModelCreate.restype = ctypes.POINTER(ctypes.c_void_p)
lib.TfLiteInterpreterCreate.restype = ctypes.POINTER(ctypes.c_void_p)
lib.TfLiteInterpreterOptionsCreate.restype = ctypes.POINTER(ctypes.c_void_p)
lib.TfLiteInterpreterOptionsSetNumThreads.argtypes = [ctypes.POINTER(ctypes.c_void_p), ctypes.c_int]
lib.TfLiteInterpreterOptionsDelete.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
lib.TfLiteInterpreterDelete.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
lib.TfLiteModelDelete.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
lib.TfLiteInterpreterGetInputTensor.restype = ctypes.POINTER(ctypes.c_void_p)
lib.TfLiteInterpreterGetOutputTensor.restype = ctypes.POINTER(ctypes.c_void_p)
lib.TfLiteTensorCopyFromBuffer.argtypes = [ctypes.POINTER(ctypes.c_void_p), ctypes.c_void_p, ctypes.c_size_t]
lib.TfLiteTensorCopyToBuffer.argtypes = [ctypes.POINTER(ctypes.c_void_p), ctypes.c_void_p, ctypes.c_size_t]
lib.TfLiteInterpreterInvoke.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
lib.TfLiteInterpreterAllocateTensors.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
lib.TfLiteInterpreterGetInputTensor.argtypes = [ctypes.POINTER(ctypes.c_void_p), ctypes.c_int]
lib.TfLiteInterpreterGetOutputTensor.argtypes = [ctypes.POINTER(ctypes.c_void_p), ctypes.c_int]

model_path = b"tflite_models/cry_detection_model_quant.tflite"
with open(model_path, 'rb') as f:
    model_data = f.read()

model = lib.TfLiteModelCreate(ctypes.c_char_p(model_data), ctypes.c_size_t(len(model_data)))

# Create interpreter options and set number of threads
options = lib.TfLiteInterpreterOptionsCreate()
lib.TfLiteInterpreterOptionsSetNumThreads(options, 2)

# Create the interpreter with the custom options
interpreter = lib.TfLiteInterpreterCreate(model, options)

# Allocate tensors
status = lib.TfLiteInterpreterAllocateTensors(interpreter)

# Get input and output tensor pointers
input_tensor = lib.TfLiteInterpreterGetInputTensor(interpreter, 0)
output_tensor = lib.TfLiteInterpreterGetOutputTensor(interpreter, 0)

def compute_mfcc(y, sr, n_mfcc=NUM_MFCC):
    # Pre-emphasis filter
    pre_emphasis = 0.97
    emphasized_signal = np.append(y[0], y[1:] - pre_emphasis * y[:-1])
    
    # Frame parameters
    frame_size = 0.025  # 25 ms
    frame_stride = 0.010  # 10 ms
    frame_length, frame_step = int(round(frame_size * sr)), int(round(frame_stride * sr))
    signal_length = len(emphasized_signal)
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step)) + 1
    
    # Padding
    pad_signal_length = (num_frames -1) * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(emphasized_signal, z)
    
    # Framing
    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + \
              np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]
    
    # Windowing with Hamming window
    frames *= np.hamming(frame_length)
    
    # Fourier Transform and Power Spectrum
    NFFT = 512
    mag_frames = np.absolute(np.fft.rfft(frames, NFFT))
    pow_frames = ((1.0 / NFFT) * (mag_frames ** 2))
    
    # Filter Banks
    nfilt = 40
    low_freq_mel = 0
    high_freq_mel = (2595 * np.log10(1 + (sr / 2) / 700))
    mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)
    hz_points = (700 * (10 ** (mel_points / 2595) - 1))
    bin = np.floor((NFFT + 1) * hz_points / sr).astype(int)
    
    fbank = np.zeros((nfilt, NFFT // 2 + 1))
    for m in range(1, nfilt + 1):
        f_m_minus = bin[m - 1]
        f_m = bin[m]
        f_m_plus = bin[m + 1]
        
        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
    
    filter_banks = np.dot(pow_frames, fbank.T)
    # Numerical stability
    filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)
    filter_banks = 20 * np.log10(filter_banks)
    
    # MFCCs
    mfcc = scipy.fftpack.dct(filter_banks, type=2, axis=1, norm='ortho')[:, :n_mfcc]
    # Mean normalization
    mfcc -= (np.mean(mfcc, axis=0) + 1e-8)
    
    return mfcc

def preprocess_audio(file_path, num_mfcc=NUM_MFCC, max_length=99, is_lstm=False):
    y, sr = sf.read(file_path)
    y = y.astype(np.float32)
    y = y / np.max(np.abs(y))
    mfcc = compute_mfcc(y, sr, n_mfcc=num_mfcc)
    
    # Pad or truncate to fixed length
    if mfcc.shape[0] < max_length:
        pad_width = ((0, max_length - mfcc.shape[0]), (0, 0))
        mfcc = np.pad(mfcc, pad_width, mode='constant')
    else:
        mfcc = mfcc[:max_length, :]
    
    if not is_lstm:
        mfcc = mfcc[..., np.newaxis]  # Add channel dimension for CNN
    return mfcc

def predict(file_path, num_mfcc=NUM_MFCC, max_length=99, is_lstm=False):
    input_data = preprocess_audio(file_path, num_mfcc, max_length, is_lstm)
    
    # Adjust input shape for LSTM and CNN
    if is_lstm:
        input_data = np.expand_dims(input_data, axis=0).astype(np.float32)  # Shape: (1, time_steps, num_mfcc)
    else:
        input_data = np.expand_dims(input_data, axis=0).astype(np.float32)  # Shape: (1, time_steps, num_mfcc, 1)
    
    # Set the tensor to point to the input data to be inferred
    lib.TfLiteTensorCopyFromBuffer(
        input_tensor,
        input_data.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
        ctypes.c_size_t(input_data.nbytes)
    )
    
    # Run inference
    lib.TfLiteInterpreterInvoke(interpreter)
    
    # Extract output data
    output_size = 1
    output_data = np.empty(output_size, dtype=np.float32)
    lib.TfLiteTensorCopyToBuffer(
        output_tensor,
        output_data.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
        ctypes.c_size_t(output_data.nbytes)
    )
    
    return output_data

def process_folder(folder_path, num_mfcc=NUM_MFCC, max_length=99, is_lstm=False):
    correct_predictions = 0
    total_files = 0
    results = []
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.wav'):
            file_path = os.path.join(folder_path, file_name)
            prediction = predict(file_path, num_mfcc, max_length, is_lstm)
            prediction_label = 'Cry' if prediction > 0.5 else 'Not Cry'
            results.append((file_name, prediction_label))
            ground_truth = 'Cry' if '_cry.wav' in file_name else 'Not Cry'

            if prediction_label == ground_truth:
                correct_predictions += 1
                if prediction_label == 'Cry':
                    true_positives += 1
            else:
                if prediction_label == 'Cry':
                    false_positives += 1
                elif prediction_label == 'Not Cry' and ground_truth == 'Cry':
                    false_negatives += 1

            total_files += 1

    # Calculate metrics
    accuracy = (correct_predictions / total_files) * 100 if total_files > 0 else 0
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    false_negative_percentage = (false_negatives / (false_negatives + true_positives)) * 100 if (false_negatives + true_positives) > 0 else 0

    return results, accuracy, f1, false_negative_percentage

# Specify the folder containing test audio files
folder_path = '{0}/Test_augmented'.format(AUDIO_PATH)
if MODEL == 'lstm':
    is_lstm_model = True
else:
    is_lstm_model = False

# Set max_length based on training parameters (adjust as necessary)
max_length = 499  # This should match the max_length used during model training

predictions, accuracy, f1_score, false_negative_percentage = process_folder(
    folder_path,
    num_mfcc=NUM_MFCC,
    max_length=max_length,
    is_lstm=is_lstm_model
)

for file_name, prediction_label in predictions:
    print(f"File: {file_name}, Prediction: {prediction_label}")

print(f"Prediction Accuracy: {accuracy:.2f}%")
print(f"F1 Score: {f1_score:.2f}")
print(f"False Negative Percentage: {false_negative_percentage:.2f}%")

# Clean up resources
lib.TfLiteInterpreterDelete(interpreter)
lib.TfLiteInterpreterOptionsDelete(options)
lib.TfLiteModelDelete(model)

print("All operations completed successfully.")


File: P20_835_cry.wav, Prediction: Cry
File: P29_2038_cry.wav, Prediction: Cry
File: P29_2304_cry.wav, Prediction: Cry
File: P29_348_cry.wav, Prediction: Cry
File: P19_520_cry.wav, Prediction: Cry
File: P29_52_cry.wav, Prediction: Cry
File: P20_893_cry.wav, Prediction: Cry
File: P34_535_notcry.wav, Prediction: Not Cry
File: P34_99_notcry.wav, Prediction: Cry
File: P29_38_cry.wav, Prediction: Cry
File: P20_599_cry.wav, Prediction: Cry
File: P34_851_notcry.wav, Prediction: Not Cry
File: P29_1201_cry.wav, Prediction: Cry
File: P29_35_cry.wav, Prediction: Cry
File: P29_1835_cry.wav, Prediction: Cry
File: P34_1227_notcry.wav, Prediction: Cry
File: P12_218_notcry.wav, Prediction: Not Cry
File: P34_434_notcry.wav, Prediction: Cry
File: P17_41_cry.wav, Prediction: Not Cry
File: P29_2442_cry.wav, Prediction: Cry
File: P34_194_notcry.wav, Prediction: Not Cry
File: P29_1483_cry.wav, Prediction: Cry
File: P29_1332_cry.wav, Prediction: Cry
File: P29_2152_cry.wav, Prediction: Cry
File: P29_564_cry.w

In [ ]:
# Trial 3 Complete [00h 02m 28s]
# val_accuracy: 0.9020833373069763

# Best val_accuracy So Far: 0.9354166388511658
# Total elapsed time: 00h 07m 32s

# Search: Running Trial #4

# Value             |Best Value So Far |Hyperparameter
# 0.0013197         |4.6117e-05        |l2
# 64                |128               |filters_1
# 0.3               |0.2               |dropout_1
# 128               |256               |filters_2
# 0.2               |0.2               |dropout_2
# 384               |128               |filters_3
# 0.4               |0.4               |dropout_3
# 256               |256               |dense_units
# 0.4               |0.3               |dropout_fc
# 0.0049677         |2.53e-05          |learning_rate